# Nomogram for a machine learning model with categorical predictors to predict binary outcome
**Author: Herdiantri Sufriyana, Emily Chia-Yu Su**  
*Date: 2024-01-15*

## Programming environment

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from joblib import dump, parallel_backend, load
import os
import shap

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
seed=2024-1-15

print(f'Seed: {seed}')

Seed: 2008


## Input dataset

In [3]:
datasets=[
    'training'
    ,'validation'
    ,'test'
]

## Training configuration

In [4]:
# Algorithms to be used
algorithms={
    'rr':LogisticRegression(penalty='l2',solver='saga',random_state=seed,max_iter=1000)
    ,'rf':RandomForestClassifier(random_state=seed)
    ,'gb':XGBClassifier(eval_metric='logloss',random_state=seed)
}

In [5]:
# Define hyperparameter grids for each algorithm
hyperparam_grids={
    'rr':{
        'C':[0.01,0.1,1,10]
    }
    ,'rf':{
        'n_estimators':[10,50,100]
        ,'max_depth':[None,10,20]
        ,'min_samples_split':[2,5,10]
    }
    ,'gb':{
        'n_estimators':[50,100,200]
        ,'learning_rate':[0.01,0.1,1]
        ,'max_depth':[3,4,5]
        ,'subsample':[0.8,1]
        ,'colsample_bytree':[0.8,1]
    }
}

In [6]:
# Define cross-validation with random seed
cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)

## Training and evaluation

In [7]:
# Load the training dataset
training_file=f"data/model_input/training.csv"
training_data=pd.read_csv(training_file)

# Extract features, labels, and sample weights from the training data
X_train=training_data.drop(columns=['outcome','outcome_weight'])
y_train=training_data['outcome']
sample_weights=training_data['outcome_weight']

# Loop through each algorithm
for algorithm_name,algorithm in algorithms.items():
    
    # Define the hyperparameter grid
    hyperparam_grid=hyperparam_grids[algorithm_name]
    
    # Train the model with cross-validated grid search and sample weighting
    grid_search=GridSearchCV(algorithm,hyperparam_grid,cv=cv,n_jobs=14,scoring='roc_auc')
    
    with parallel_backend('threading',n_jobs=14):
        grid_search.fit(X_train,y_train,sample_weight=sample_weights)
           
    # Get the best model and best hyperparameters from the grid search
    best_model=grid_search.best_estimator_
    best_hyperparams=grid_search.best_params_
    
    # Define the directory to save the models
    save_dir=f"data/sklearn_models/{algorithm_name}"
    
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Save the trained model to a file
    model_filename=f"{save_dir}/model.joblib"
    dump(best_model,model_filename)
    
    # Save the best hyperparameters to a CSV file
    hyperparams_df=pd.DataFrame([best_hyperparams])
    hyperparams_filename=f"{save_dir}/best_hyperparams.csv"
    hyperparams_df.to_csv(hyperparams_filename,index=False)
            
    # Loop through each prediction set and make predictions
    for dataset in datasets:
        
        prediction_file=f"data/model_input/{dataset}.csv"

        # Load the prediction dataset
        prediction_data=pd.read_csv(prediction_file)
        
        # Extract features from the prediction data
        X_pred=prediction_data.drop(columns=['outcome','outcome_weight'])
        y_true=prediction_data['outcome']
        
        # Make predictions on the prediction data
        y_pred_proba=best_model.predict_proba(X_pred)[:,1]
        
        # Output model evaluation
        print(f"Training file: {training_file}, Algorithm: {algorithm_name}, Prediction file: {prediction_file}")
        print(f"AUC-ROC: {roc_auc_score(y_true,y_pred_proba)}\n")
        
        # Write the predicted probabilities to CSV
        output_filename=f"{save_dir}/prob_{dataset}.csv"
        pd.DataFrame({'outcome':y_true,'prob':y_pred_proba}).to_csv(output_filename,index=False)

Training file: data/model_input/training.csv, Algorithm: rr, Prediction file: data/model_input/training.csv
AUC-ROC: 0.9681840268574963

Training file: data/model_input/training.csv, Algorithm: rr, Prediction file: data/model_input/validation.csv
AUC-ROC: 0.9844074844074844

Training file: data/model_input/training.csv, Algorithm: rr, Prediction file: data/model_input/test.csv
AUC-ROC: 0.9717261904761905

Training file: data/model_input/training.csv, Algorithm: rf, Prediction file: data/model_input/training.csv
AUC-ROC: 0.9729768530788938

Training file: data/model_input/training.csv, Algorithm: rf, Prediction file: data/model_input/validation.csv
AUC-ROC: 0.9792099792099792

Training file: data/model_input/training.csv, Algorithm: rf, Prediction file: data/model_input/test.csv
AUC-ROC: 0.9673763736263736

Training file: data/model_input/training.csv, Algorithm: gb, Prediction file: data/model_input/training.csv
AUC-ROC: 0.9704147892923403

Training file: data/model_input/training.csv,

In [7]:
# Corresponding explainers
explainers={
    'rr':shap.LinearExplainer
    ,'rf':shap.TreeExplainer
    ,'gb':shap.TreeExplainer
}

In [8]:
# Load the training dataset
training_file=f"data/model_input/training.csv"
training_data=pd.read_csv(training_file)

# Extract features, labels, and sample weights from the training data
X_train=training_data.drop(columns=['outcome','outcome_weight'])

# Loop through each algorithm
for algorithm_name,algorithm in algorithms.items():
    
    # Define the directory to load the trained models
    save_dir=f"data/sklearn_models/{algorithm_name}"
    
    # Load the trained model from a file
    model_filename=os.path.join(save_dir,'model.joblib')
    best_model=load(model_filename)
    
    # Initialize the SHAP explainer
    explainer=explainers[algorithm_name](best_model,X_train)
    
    # Calculate SHAP values
    shap_values=explainer.shap_values(X_train)
    
    # Check if shap_values is a list (e.g., for binary classification problems)
    if isinstance(shap_values,list):
        # Select the SHAP values for the class of interest (e.g., class 1)
        shap_values=shap_values[1]
    
    # Convert SHAP values to DataFrame and save to CSV
    shap_csv_filename=os.path.join(save_dir,'shap_values.csv')
    pd.DataFrame(shap_values,columns=X_train.columns).to_csv(shap_csv_filename,index=False)

In [10]:
# Choose algorithm
algorithm_name='rf'
algorithm=RandomForestClassifier(random_state=seed)

# Define the directory to load the trained models
save_dir=f"data/sklearn_models/{algorithm_name}"

# Load the trained model from a file
model_filename=os.path.join(save_dir,'model.joblib')
best_model=load(model_filename)

# Choose prediction file
prediction_file='data/pred_data_nomogram.csv'

# Load the prediction dataset
prediction_file=f"data/dataset_nomogram.csv"
prediction_data=pd.read_csv(prediction_file)

# Extract features from the prediction data
X_pred=prediction_data.drop(columns=['outcome','outcome_onset','outcome_weight'])

# Make predictions on the prediction data
y_pred_proba=best_model.predict_proba(X_pred)[:,1]

# Output model evaluation
print(f"Algorithm: {algorithm_name}, Prediction file: {prediction_file}")

# Write the predicted probabilities to CSV
output_filename=os.path.join(save_dir,f"prob_{os.path.basename(prediction_file)}")
pd.DataFrame({'prob':y_pred_proba}).to_csv(output_filename,index=False)

Algorithm: rf, Prediction file: data/dataset_nomogram.csv
